In [1]:
from os import getenv
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain_google_genai import ChatGoogleGenerativeAI

#### You will need [Neo4j APOC](https://neo4j.com/docs/apoc/current/installation/) for langchain_community.graph.Neo4jGraph to function.
Follow the instructions in the link and then, in neo4j.conf, set 
`dbms.security.procedures.unrestricted=apoc.*`

In [2]:
load_dotenv()

True

In [3]:
neo4j_url= getenv("NEO4J_URL")
neo4j_user = getenv("NEO4J_USER")
neo4j_password = getenv("NEO4J_PW")
neo4j_database = getenv("NEO4J_DB")

In [4]:
graph = Neo4jGraph(neo4j_url, neo4j_user, neo4j_password, neo4j_database)

In [5]:
print(graph.schema)

Node properties:
Movie {tagline: STRING, rating: FLOAT, certificate: STRING, title: STRING, run_time: INTEGER, year: INTEGER}
Person {name: STRING}
Actor {name: STRING}
Director {name: STRING}
Writer {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:BELONGS_TO]->(:Genre)
(:Person)-[:WROTE]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Actor)-[:ACTED_IN]->(:Movie)
(:Actor)-[:WROTE]->(:Movie)
(:Actor)-[:DIRECTED]->(:Movie)
(:Director)-[:WROTE]->(:Movie)
(:Director)-[:DIRECTED]->(:Movie)
(:Director)-[:ACTED_IN]->(:Movie)
(:Writer)-[:WROTE]->(:Movie)
(:Writer)-[:DIRECTED]->(:Movie)
(:Writer)-[:ACTED_IN]->(:Movie)


#### Gemini 

In [6]:
gemini_api = getenv("GEMINI_API")

In [7]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key = gemini_api ,temperature = 0)
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)

### Prompts

- #### Simple prompts.

In [8]:
questions = [
    "Which are the best movies released before 1960?",
    "Which are the 3 best crime dramas?",
    "Who is the director with highest average rating for their movies?",
    "In what movies have Robert DeNiro and Al Pacino together?",
    "Which director has Brad Pitt worked with the most?"
]

for question in questions:
    print(chain.invoke(question)["result"])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie)
WHERE m.year < 1960
RETURN m.title AS Title, m.rating AS Rating
ORDER BY m.rating DESC
LIMIT 10;

Full Context:
[{'Title': '12 Angry Men', 'Rating': 9.0}, {'Title': "It's a Wonderful Life", 'Rating': 8.6}, {'Title': 'Seven Samurai', 'Rating': 8.6}, {'Title': 'City Lights', 'Rating': 8.5}, {'Title': 'Rear Window', 'Rating': 8.5}, {'Title': 'Casablanca', 'Rating': 8.5}, {'Title': 'Modern Times', 'Rating': 8.5}, {'Title': 'Sunset Blvd.', 'Rating': 8.4}, {'Title': 'Paths of Glory', 'Rating': 8.4}, {'Title': 'The Great Dictator', 'Rating': 8.4}]

> Finished chain.
12 Angry Men, It's a Wonderful Life, Seven Samurai, City Lights, Rear Window, Casablanca, Modern Times, Sunset Blvd., Paths of Glory, The Great Dictator


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie)-[:BELONGS_TO]->(:Genre {name: "Crime"})
RETURN m.title, m.rating
ORDER BY m.rating DESC
LIMIT 3

Full Context

- #### Some prompts need clarification. In this case, writers, directors  and actors who have worked together need to be specified as different people.

In [9]:
questions = [
    "Which actor and director have worked together the most?",
    "Which writer and director have worked together the most?",

    "Which actor and director have worked together the most? Remember they cannot be the same person.",
    "Which writer and director have worked together the most? Remember they cannot be the same person."
]

for question in questions:
    print(chain.invoke(question)["result"])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(d:Director)
RETURN a.name AS actor_name, d.name AS director_name, COUNT(*) AS collaborations
ORDER BY collaborations DESC
LIMIT 1;

Full Context:
[{'actor_name': 'Charles Chaplin', 'director_name': 'Charles Chaplin', 'collaborations': 5}]

> Finished chain.
Charles Chaplin and Charles Chaplin have worked together the most, with 5 collaborations.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (w:Writer)-[:WROTE]->(m:Movie)<-[:DIRECTED]-(d:Director)
RETURN w.name AS Writer, d.name AS Director, count(m) AS Collaborations
ORDER BY Collaborations DESC
LIMIT 1;

Full Context:
[{'Writer': 'Christopher Nolan', 'Director': 'Christopher Nolan', 'Collaborations': 7}]

> Finished chain.
Christopher Nolan has worked with himself the most.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)<-[:

- #### Sometimes the LLM generates proper Cypher queries, which return the proper results, but the LLM is unable to infer the required information from these results.
- #### Adding another line that motivates it to understand helps in these cases.

In [10]:
questions = [
    "Who is the actor who has starred in most of the best rated movies.",
    
    "Who is the actor who has starred in most of the best rated movies." \
    "You know the answer. Just parse the result from the Cypher query, understand it, then tell me the actor",
]

for question in questions:
    print(chain.invoke(question)["result"])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)
WHERE m.rating > 4.0
RETURN a.name, COUNT(m) AS num_movies
ORDER BY num_movies DESC
LIMIT 1;

Full Context:
[{'a.name': 'Robert De Niro', 'num_movies': 9}]

> Finished chain.
I don't know the answer to this question.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)
WHERE m.rating > 4.0
RETURN a.name, count(m) AS num_movies
ORDER BY num_movies DESC
LIMIT 1;

Full Context:
[{'a.name': 'Robert De Niro', 'num_movies': 9}]

> Finished chain.
Robert De Niro has starred in 9 movies.


### Enter your own prompts -

In [12]:
while True:
    question = input("\n\nEnter your question (Type 'EXIT' to exit) : ")
    
    if question == "EXIT":
        print("Exiting")
        break
    else:
        print(chain.invoke(question)["result"])



Enter your question (Type 'EXIT' to exit) :  Top 5 thriller movies made before 2000.




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie)-[:BELONGS_TO]->(:Genre {name: "Thriller"})
WHERE m.year < 2000
RETURN m.title, m.year
ORDER BY m.rating DESC
LIMIT 5;

Full Context:
[{'m.title': 'The Silence of the Lambs', 'm.year': 1991}, {'m.title': 'Rear Window', 'm.year': 1954}, {'m.title': 'Psycho', 'm.year': 1960}, {'m.title': 'Come and See', 'm.year': 1985}, {'m.title': 'Reservoir Dogs', 'm.year': 1992}]

> Finished chain.
1. The Silence of the Lambs (1991)
2. Rear Window (1954)
3. Psycho (1960)
4. Come and See (1985)
5. Reservoir Dogs (1992)




Enter your question (Type 'EXIT' to exit) :  Best Robert DeNiro movie?




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie)<-[:ACTED_IN]-(a:Actor {name: "Robert De Niro"})
RETURN m.title AS BestMovie, m.rating AS Rating
ORDER BY m.rating DESC
LIMIT 1;

Full Context:
[{'BestMovie': 'The Godfather Part II', 'Rating': 9.0}]

> Finished chain.
I don't know the answer to that question.




Enter your question (Type 'EXIT' to exit) :  Best Robert DeNiro movie? You know the answer. Parse the result and answer.




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)<-[:ACTED_IN]-(a:Actor {name: "Robert De Niro"})
RETURN m.title AS best_movie_by_Robert_De_Niro
ORDER BY m.rating DESC
LIMIT 1
Full Context:
[{'best_movie_by_Robert_De_Niro': 'The Godfather Part II'}]

> Finished chain.
The Godfather Part II




Enter your question (Type 'EXIT' to exit) :  Which actor has Robert DeNiro worked alongside the most?




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Actor {name: "Robert De Niro"})-[:ACTED_IN]-(m:Movie)
WITH m, a
MATCH (m)-[:ACTED_IN]-(co:Actor)
WHERE co <> a
RETURN co.name AS Co_Actor, COUNT(*) AS Number_of_Collaborations
ORDER BY Number_of_Collaborations DESC
LIMIT 1;

Full Context:
[{'Co_Actor': 'Joe Pesci', 'Number_of_Collaborations': 3}]

> Finished chain.
Joe Pesci has worked alongside Robert DeNiro the most, with 3 collaborations.




Enter your question (Type 'EXIT' to exit) :  EXIT


Exiting
